In [ ]:
import torch
import sys
sys.path.append("/DRIVEcodeV1/DCG_Core")
sys.path.append("/your/dataset/path")
import pytorch_lightning as pl
# from module_copy1 import *
from tqdm import tqdm
import os


In [2]:
#checkpoints path& task setup

task = "angle"#angle/distance/multitask ,align with version path
version_path="/ckpt_final/../version1"
prediction_save_path ="../predictions"
# text_save_rootpath =f"/hpc2hdd/home/tianlangxue/XAI4AD/concept_gridlock/New/results/MAEtext/{task}_{os.path.basename(version_path)}.txt"
# ckpts_path =version_path+"/checkpoints"
# ckpt_path_list = get_ckpt_files(ckpts_path)

In [ ]:

# from model import *
# from module import  *
from module_copy1_only4conceptpredict import  *
from model_copy1 import *
from model_copy2 import *
from main_copy1 import save_preds

teachmodel=TVTN(multitask=task, backbone="none", concept_features=True, device = f"cuda:0", train_concepts=False)
model = VTN(multitask=task, backbone= "none", concept_features=True, device = f"cuda:0", train_concepts=False)
# module = LaneModule(model=model,teachmodel=teachmodel, multitask=task, dataset = "comma", bs=2, ground_truth="normal", intervention=False, img_noise=args.img_noise,dataset_path=args.dataset_path, dataset_fraction=args.dataset_fraction)
# checkpoint = torch.load(ckpt_path_list[1])


In [4]:
# testmodule.model.state_dict
# testmodule.teachmodel.state_dict

In [ ]:
img_noise=None
MAE_list=[]#save num of epochs mae results
# os.makedirs(text_save_rootpath, exist_ok=True)
# from dataloader_comma import *
# if not os.path.exists(prediction_save_path):
#     os.makedirs(prediction_save_path)
gn1_DRIVE_ckpt_path= "/hpc2hdd/home/tianlangxue/XAI4AD/comma2k19data/ckpts_final/ckpts_final_comma_angle_none_True_1/lightning_logs/version_119_40epoch_5loss/checkpoints/epoch=34-step=4200_GNV1.ckpt"
gn1_DCG_ckpt_path ="/hpc2hdd/home/tianlangxue/XAI4AD/comma2k19data/ckpts_final/ckpts_final_comma_angle_none_True_1/lightning_logs/version_20_8.23/checkpoints/epoch=185-step=11160_GNV2.ckpt"
clean_DRIVE_ckpt_path= "/hpc2hdd/home/tianlangxue/XAI4AD/comma2k19data/ckpts_final/ckpts_final_comma_angle_none_True_1/lightning_logs/version_119_40epoch_5loss/checkpoints/epoch=34-step=4200.ckpt"

clean_DCG_ckpt_path="/hpc2hdd/home/tianlangxue/XAI4AD/comma2k19data/ckpts_final/ckpts_final_comma_angle_none_True_1/lightning_logs/version_20_8.23/checkpoints/epoch=185-step=11160.ckpt"
# for id in tqdm(range(len(ckpt_path_list))):
    
# print(os.path.basename(ckpt_path_list[id]).split("-")[0])

# define modules using different inputs
perturbed_module = LaneModule.load_from_checkpoint(
    checkpoint_path=gn1_DRIVE_ckpt_path,
    model=model, 
    teachmodel =teachmodel,
    multitask=task,
    dataset = "comma",
    dataset_path="/hpc2hdd/home/tianlangxue/XAI4AD/comma2k19data",
    bs=1,  
    ground_truth="desired",
    # intervention=args.intervention_prediction,
    img_noise=img_noise,
    # dataset_path=args.dataset_path,
    # dataset_fraction=args.dataset_fraction
    )

clean_module = LaneModule.load_from_checkpoint(
    checkpoint_path=clean_DRIVE_ckpt_path,
    model=model,
    teachmodel =teachmodel, 
    multitask=task,
    dataset = "comma",
    dataset_path="/hpc2hdd/home/tianlangxue/XAI4AD/comma2k19data",
    bs=1,  
    ground_truth="desired",
    # intervention=args.intervention_prediction,
    img_noise=None,
    # dataset_path=args.dataset_path,
    # dataset_fraction=args.dataset_fraction
    )

trainer = pl.Trainer(
    gpus=1,
    accelerator='gpu',
    devices="cuda:0",

)

perturbedpreds =trainer.predict(perturbed_module)
cleanpreds = trainer.predict(clean_module)
# assert len(perturbedpreds) == len(cleanpreds), "The lengths of perturbedpreds and cleanpreds should be the same."






In [ ]:
output_file = "DRIVE_ParamGNV1_top5overlap.txt"

total_overlap_rate = []
with open(output_file, 'w') as file:
    for cleanpred, ptbpred in zip(cleanpreds, perturbedpreds):
        clean_concept = cleanpred[3]  
        ptb_concept = ptbpred[3]      
        

        clean_top5_indices = torch.tensor(clean_concept.squeeze()).topk(5, dim=-1).indices  # size [240,5]
    
        ptb_top5_indices = torch.tensor(ptb_concept.squeeze()).topk(5, dim=-1).indices  # size [240,5]
        print(clean_top5_indices.shape)
        print(ptb_top5_indices.shape)
        num_frames, top_k = clean_top5_indices.shape
        frame_overlap_rates = []

        frame_rates = []
        print(range(num_frames))
        for f in list(range(num_frames)):
            print(clean_top5_indices[f].tolist())
            clean_set = set(clean_top5_indices[f].tolist())
            ptb_set = set(ptb_top5_indices[f].tolist())
            overlap = len(clean_set.intersection(ptb_set))
            total = min(len(clean_set), len(ptb_set))
            if total > 0:
                rate = overlap / total
                frame_rates.append(rate)
            else:
                frame_rates.append(0.0)
        frame_overlap_rates.append(frame_rates)  

     
        avg_frame_overlap_rate = [sum(frame_rate) / len(frame_rate) for frame_rate in frame_overlap_rates]

    
        total_overlap_rate.extend(avg_frame_overlap_rate)

        file.write(f"Sample average overlap rates: {avg_frame_overlap_rate}\n")

    if total_overlap_rate:
        overall_avg_overlap_rate = sum(total_overlap_rate) / len(total_overlap_rate)
    else:
        overall_avg_overlap_rate = 0.0
    file.write(f"Overall average overlap rate: {overall_avg_overlap_rate}\n")